In [10]:
#load the twiter data
#data taken from https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter?resource=download&select=twcs and reduced it 
#to work with my system limitation
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('files/twcs.csv')

# Print the DataFrame
df['text']

0      @115712 I understand. I would like to assist y...
1          @sprintcare and how do you propose we do that
2      @sprintcare I have sent several private messag...
3      @115712 Please send us a Private Message so th...
4                                     @sprintcare I did.
                             ...                        
400    @AppleSupport why are my I️’s changing not sho...
401    @115855 Let's go to DM for the next steps. DM ...
402    @AppleSupport Tried resetting my settings .. r...
403    @115855 Any steps tried since it started last ...
404    @AppleSupport This is what it looks like https...
Name: text, Length: 405, dtype: object

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Tokenize the text
tokens = word_tokenize(df['text'])

# Define a list of stop words
stop_words = set(stopwords.words("english"))

# Define a list of punctuation
punctuation = set(string.punctuation)

# Define a list to store the filtered tokens
filtered_tokens = []

# Iterate over the tokens
for token in tokens:
    # Check if the token is not a stop word or punctuation
    if token not in stop_words and token not in punctuation:
        filtered_tokens.append(token)

# Print the filtered tokens
print(filtered_tokens)


AttributeError: 'str' object has no attribute 'str'

In [13]:
from collections import defaultdict
import pandas as pd
import nltk

# Tokenize the text corpus
tokens = nltk.word_tokenize(df.text)

# Create a defaultdict to store the co-occurrence counts
co_occur = defaultdict(lambda: defaultdict(int))

# Define the window size
window_size = 2

# Iterate over the tokens
for i, word in enumerate(tokens):
    # Iterate over the words within the window size
    for j in range(i-window_size, i+window_size+1):
        # Check if the index is within the range of tokens
        if j >= 0 and j < len(tokens) and j != i:
            # Increment the co-occurrence count
            co_occur[word][tokens[j]] += 1

# Convert the defaultdict to a dataframe
co_occur_df = pd.DataFrame(co_occur)

# Print the co-occurrence matrix
co_occur_df


TypeError: expected string or bytes-like object, got 'Series'

In [26]:
import nltk
import numpy as np
from collections import defaultdict
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

# Define the text corpus
text = "I love to play football. Football is a great game. I also love to play soccer. Soccer is a great game too. I love to play all kinds of sports."

# Tokenize the text corpus
tokens = nltk.word_tokenize(text)

# Create a BigramCollocationFinder object
finder = BigramCollocationFinder.from_words(tokens)

# Define the measure to use (pointwise mutual information)
pmi_measure = BigramAssocMeasures.pmi

# Compute the co-occurrence matrix
co_occur_matrix = finder.score_ngrams(pmi_measure)

# Create a defaultdict to store co-occurrence counts
co_occur = defaultdict(lambda: defaultdict(int))

# Extract the co-occurrence counts from the tuples
for bigram, count in co_occur_matrix:
    w1, w2 = bigram
    co_occur[w1][w2] = count

# Convert the defaultdict to a 2D numpy array
co_occur_matrix = np.array([[co_occur[w1][w2] for w2 in co_occur] for w1 in co_occur])

# Compute the marginal probabilities for each word
p_w1 = np.sum(co_occur_matrix, axis=1) / np.sum(co_occur_matrix)
p_w2 = np.sum(co_occur_matrix, axis=0) / np.sum(co_occur_matrix)

# Compute the pointwise mutual information for each word pair
pmi = co_occur_matrix * np.log2(co_occur_matrix / (p_w1[:, None] * p_w2))

# Compute PPMI by setting negative values to 0
co_occur_matrix_ppmi = np.maximum(pmi, 0)

# Print the PPMI matrix
print(co_occur_matrix_ppmi)


[[        nan 54.30088097         nan         nan         nan         nan
          nan         nan         nan         nan         nan         nan
          nan         nan         nan         nan         nan         nan
          nan]
 [        nan         nan 54.30088097         nan         nan         nan
          nan         nan         nan         nan         nan         nan
          nan         nan         nan         nan         nan         nan
          nan]
 [        nan         nan         nan         nan         nan         nan
          nan         nan         nan         nan         nan         nan
          nan         nan         nan         nan         nan 54.30088097
          nan]
 [        nan         nan         nan         nan         nan         nan
          nan         nan 40.87706305         nan         nan         nan
          nan         nan         nan         nan         nan         nan
          nan]
 [        nan         nan         nan         nan   

C:\Users\ranjit09\AppData\Local\Temp\ipykernel_8504\591510656.py:37: RuntimeWarning: divide by zero encountered in log2
  pmi = co_occur_matrix * np.log2(co_occur_matrix / (p_w1[:, None] * p_w2))
C:\Users\ranjit09\AppData\Local\Temp\ipykernel_8504\591510656.py:37: RuntimeWarning: invalid value encountered in multiply
  pmi = co_occur_matrix * np.log2(co_occur_matrix / (p_w1[:, None] * p_w2))


In [28]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

#convert the NaN value to 0, as SVD will not be able to handle NaN
co_occur_matrix_ppmi[np.isnan(co_occur_matrix_ppmi)] = 0

# Perform SVD on the PPMI matrix
svd = TruncatedSVD(n_components=2) #no of dimension=2
word_embeddings = svd.fit_transform(co_occur_matrix_ppmi)

# Print the word embeddings
print(word_embeddings)


[[ 1.79055383e-02 -7.55247056e-02]
 [ 6.14759960e-02 -2.60373098e-01]
 [ 1.20620587e-01 -5.55697239e-01]
 [ 2.00103530e-01  4.08635661e+01]
 [ 2.00103530e-01  4.08635661e+01]
 [ 1.19230115e-01 -4.71440150e-01]
 [-8.53881881e-02  2.89919248e-01]
 [ 4.11345326e-02 -1.28816453e-01]
 [-4.76530950e-02  1.80667297e-01]
 [ 5.79978090e-02 -2.55592666e-01]
 [ 3.90303962e-02 -1.80961858e-01]
 [ 9.22118039e-02 -2.93483299e-01]
 [ 5.85313438e+01 -2.77359206e-01]
 [ 7.24669251e-02 -3.98586409e-01]
 [-6.84343241e-02  2.68519584e-01]
 [ 5.02169105e-02 -2.01276325e-01]
 [ 5.02169105e-02 -2.01276325e-01]
 [ 5.02169105e-02 -2.01276325e-01]
 [ 5.02169105e-02 -2.01276325e-01]]
